Slooze Take-Home Challenge — Inventory, Purchase, and Sales Optimization  
 
Objective: Analyze and optimize inventory management for a retail wine & spirits company operating across multiple locations.

---

## Project Overview

This notebook delivers a data-driven analysis of sales, purchases, and inventory records to extract actionable business insights and optimize stock management efficiency.  
The analysis leverages historical datasets to forecast demand, identify top-performing products, and refine supplier performance — enabling better decision-making for procurement and stocking.


### 1️⃣ Demand Forecasting
- Analyze historical sales data to predict future demand.
- Use time-series models or moving averages for trend estimation.

### 2️⃣ ABC Analysis
- Classify inventory into:
  - A — High-value items (top ~80% of revenue)  
  - B — Moderate-value items  
  - C — Low-priority items  
- Prioritize A-class products for tighter control.

### 3️⃣ Economic Order Quantity (EOQ) Analysis
- Calculate the optimal order quantity (EOQ) that minimizes total cost (ordering + carrying).
- Use EOQ formula:  
  \[
  EOQ = \sqrt{\frac{2DS}{H}}
  \]  
  where  
  *D = annual demand, S = cost per order, H = holding cost per unit/year.*

### 4️⃣ Reorder Point (ROP) Analysis
- Determine reorder levels to prevent stockouts.  
- Incorporate average lead time and safety stock into ROP:  
  \[
  ROP = (\text{Average Daily Demand} \times \text{Lead Time}) + \text{Safety Stock}
  \]

### 5️⃣ Lead Time Analysis
- Calculate and optimize supplier lead times using purchase data.  
- Identify fastest and slowest suppliers to improve procurement planning and reduce delays.

---

## Expected Outcomes
 Optimized inventory control levels  
 Better supplier reliability and ordering schedules  
 Reduced carrying cost and financial inefficiency  
 Improved forecast accuracy and process visibility  

In [ ]:
!pip install kagglehub pandas numpy matplotlib seaborn scipy statsmodels prophet xgboost


In [ ]:
import kagglehub


path = kagglehub.dataset_download("sloozecareers/slooze-challenge")

print("Path to dataset files:", path)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
base_path = "/Users/teens/Desktop/challenge"

sales = pd.read_csv(os.path.join(base_path, "SalesFINAL12312016.csv"))
purchases = pd.read_csv(os.path.join(base_path, "PurchasesFINAL12312016.csv"))
invoice_purchases = pd.read_csv(os.path.join(base_path, "InvoicePurchases12312016.csv"))
begin_inventory = pd.read_csv(os.path.join(base_path, "BegInvFINAL12312016.csv"))
end_inventory = pd.read_csv(os.path.join(base_path, "EndInvFINAL12312016.csv"))
purchase_prices = pd.read_csv(os.path.join(base_path, "2017PurchasePricesDec.csv"))

print("datasets loaded ")


In [ ]:
#to see cloumns
datasets = {
    "Sales": sales,
    "Purchases": purchases,
    "InvoicePurchases": invoice_purchases,
    "BeginInventory": begin_inventory,
    "EndInventory": end_inventory,
    "PurchasePrices": purchase_prices
}

for name, df in datasets.items():
    print(f"\n {name} columns:")
    print(df.columns.tolist())


In [ ]:
#identify missing values
for name, df in datasets.items():
    print(f"\n{name} missing values:\n", df.isnull().sum().sort_values(ascending=False).head(5))


In [ ]:
print("Sales columns:\n", sales.columns.tolist())

In [ ]:
# Calculating revenue
sales['Revenue'] = sales['SalesQuantity'] * sales['SalesPrice']

sales[['InventoryId', 'Brand', 'Description', 'SalesQuantity', 'SalesPrice', 'Revenue']].head()


In [ ]:
top_stores = sales.groupby('Store')['Revenue'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(10,5))
sns.barplot(x=top_stores.values, y=top_stores.index, palette='rocket')
plt.title(" Top 10 Stores by Total Revenue")
plt.xlabel("Total Revenue ($)")
plt.ylabel("Store")
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# adjust the path to where the CSV actually lives
base_path = "/Users/Teens/Desktop/challenge"

sales = pd.read_csv(f"{base_path}/SalesFINAL12312016.csv")



In [ ]:
sales.head(2)

In [ ]:
sales["Revenue"] = sales["SalesQuantity"] * sales["SalesPrice"]
sales.head(2)



In [ ]:
top_products = (
    sales.groupby("Description")["Revenue"]
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

plt.figure(figsize=(10, 5))
sns.barplot(
    x=top_products.values,
    y=top_products.index,
    hue=top_products.index,   # fixes FutureWarning
    palette="viridis",
    legend=False,
)
plt.title("Top 10 Products by Revenue")
plt.xlabel("Total Revenue ($)")
plt.ylabel("Product Description")
plt.tight_layout()
plt.show()


In [ ]:
# Group data by Store, sum the total Revenue for each store, and get the top 10
top_stores = (
    sales.groupby("Store")["Revenue"]
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

# Plot Top 10 Stores by Total Revenue
plt.figure(figsize=(10,5))
sns.barplot(
    x=top_stores.values,
    y=top_stores.index,
    hue=top_stores.index,
    palette='rocket',
    legend=False
)
plt.title("Top 10 Stores by Total Revenue")
plt.xlabel("Total Revenue ($)")
plt.ylabel("Store")
plt.tight_layout()
plt.show()


In [ ]:
# SalesDate to datetime type
sales["SalesDate"] = pd.to_datetime(sales["SalesDate"], errors="coerce")

monthly_sales = (
    sales.groupby(pd.Grouper(key="SalesDate", freq="ME"))["SalesQuantity"]
    .sum()
    .reset_index()
)

# monthly sales trend
plt.figure(figsize=(8,4))
sns.lineplot(data=monthly_sales, x="SalesDate", y="SalesQuantity", marker="o", color="teal")
plt.title("Monthly Sales (Jan–Feb 2016)")
plt.xlabel("Month")
plt.ylabel("Total Quantity Sold")
plt.grid(True)
plt.tight_layout()
plt.show()

monthly_sales["Forecast"] = monthly_sales["SalesQuantity"].rolling(window=3).mean()
monthly_sales.tail()


In [ ]:
# Group total revenue for each product
total_revenue = (
    sales.groupby("Description")["Revenue"]
    .sum()
    .sort_values(ascending=False)
)

# Calculate cumulative percentage
cum_percent = 100 * total_revenue.cumsum() / total_revenue.sum()

# ABC DataFrame
abc_df = pd.DataFrame({
    "Revenue": total_revenue,
    "Cumulative%": cum_percent
})

# Classify products to A/B/C categories
abc_df["Category"] = pd.cut(
    cum_percent,
    bins=[0, 80, 95, 100],   # A = top 80%, B = next 15%, C = last 5%
    labels=["A", "B", "C"]
)

# count of each category
abc_df["Category"].value_counts()


In [ ]:
# product per category
for cat in ["A", "B", "C"]:
    print(f"\nTop 5 products in Category {cat}:")
    display(abc_df[abc_df["Category"] == cat].head(5))


In [ ]:

A_items = abc_df[abc_df["Category"] == "A"]
B_items = abc_df[abc_df["Category"] == "B"]
C_items = abc_df[abc_df["Category"] == "C"]

# top 10 A-class high value items
A_items.sort_values("Revenue", ascending=False).head(10)


In [ ]:
import numpy as np

# EOQ = sqrt( (2 * D * S) / H )
# D = annual demand (total quantity sold)
# S = ordering cost per order (assumed or estimated)
# H = holding cost per unit per year (assumed or estimated)

D = sales["SalesQuantity"].sum()  
S = 50                             
H = 2                              

EOQ = np.sqrt((2 * D * S) / H)

print(" EOQ (Optimal Order Quantity):", round(EOQ, 2), "units")


In [ ]:
print("Purchases columns:\n", purchases.columns.tolist())


In [ ]:
# Convert relevant date columns to datetime
purchases["PODate"] = pd.to_datetime(purchases["PODate"], errors="coerce")
purchases["ReceivingDate"] = pd.to_datetime(purchases["ReceivingDate"], errors="coerce")

# Calculate Lead Time in days
purchases["LeadTime_Days"] = (purchases["ReceivingDate"] - purchases["PODate"]).dt.days

# Filter out any unrealistic or missing lead times
purchases = purchases[purchases["LeadTime_Days"].between(0, 60)]  # assuming typical range within 2 months

# Compute average and standard deviation of lead time
avg_lead_time = purchases["LeadTime_Days"].mean()
std_lead_time = purchases["LeadTime_Days"].std()

# Compute average daily demand from Sales
daily_demand = sales["SalesQuantity"].sum() / 365

# Safety stock (for 95% service level, z = 1.65)
z = 1.65
safety_stock = z * std_lead_time * (daily_demand ** 0.5)

# Reorder Point (ROP)
ROP = (daily_demand * avg_lead_time) + safety_stock

print(f"Average Lead Time: {avg_lead_time:.2f} days")
print(f"  Safety Stock: {round(safety_stock)} units")
print(f" Reorder Point (ROP): {round(ROP)} units")


In [ ]:
# Group by VendorName to measure efficiency
supplier_lead = (
    purchases.groupby("VendorName")["LeadTime_Days"]
    .agg(["mean", "std", "count"])
    .reset_index()
    .sort_values("mean")
)

# Display top 10 fastest suppliers
print(" Fastest 10 Suppliers (Lowest Average Lead Time):")
display(supplier_lead.head(10))

# Display slowest 10 suppliers
print("\nSlowest 10 Suppliers (Highest Average Lead Time):")
display(supplier_lead.tail(10))


In [ ]:
# pick the top 10 fastest and slowest for comparison
fastest = supplier_lead.head(10)
slowest = supplier_lead.tail(10)

fig, ax = plt.subplots(1, 2, figsize=(14,5))

# Fastest suppliers
sns.barplot(x="mean", y="VendorName", data=fastest, palette="Greens_r", ax=ax[0])
ax[0].set_title("Fastest Suppliers (Lowest Avg Lead Time)")
ax[0].set_xlabel("Average Lead Time (Days)")
ax[0].set_ylabel("Vendor")

# Slowest suppliers
sns.barplot(x="mean", y="VendorName", data=slowest, palette="Reds", ax=ax[1])
ax[1].set_title("Slowest Suppliers (Highest Avg Lead Time)")
ax[1].set_xlabel("Average Lead Time (Days)")
ax[1].set_ylabel("Vendor")

plt.tight_layout()
plt.show()


In [ ]:
print("Begin Inventory Columns:\n", begin_inventory.columns.tolist())
print("\nEnd Inventory Columns:\n", end_inventory.columns.tolist())


In [ ]:
# Merge beginning and ending inventories using common identifiers
merged_inv = begin_inventory.merge(
    end_inventory,
    on=["InventoryId", "Store", "Brand", "Description"],
    suffixes=("_Beg", "_End")
)

# Ensure the stock numbers are numeric
merged_inv["onHand_Beg"] = pd.to_numeric(merged_inv["onHand_Beg"], errors="coerce")
merged_inv["onHand_End"] = pd.to_numeric(merged_inv["onHand_End"], errors="coerce")

# Calculate change in stock
merged_inv["ChangeInStock"] = merged_inv["onHand_End"] - merged_inv["onHand_Beg"]

# Fast-moving (stock dropped) and slow-moving (stock rose) items
fast_movers = merged_inv.sort_values("ChangeInStock").head(10)
slow_movers = merged_inv.sort_values("ChangeInStock", ascending=False).head(10)

print(" Fast-Moving Products (Largest Stock Decrease):")
display(fast_movers[["InventoryId","Brand","Description","onHand_Beg","onHand_End","ChangeInStock"]])

print("\n Slow-Moving / Overstocked Products (Largest Stock Increase):")
display(slow_movers[["InventoryId","Brand","Description","onHand_Beg","onHand_End","ChangeInStock"]])


In [ ]:
# Take top 10 fastest movers and slowest movers from previous cell
top_fast = fast_movers.sort_values("ChangeInStock").head(10)
top_slow = slow_movers.sort_values("ChangeInStock", ascending=False).head(10)

# Create side-by-side plots
fig, ax = plt.subplots(1, 2, figsize=(15,5))

# Fast-Moving Items (stock decreased)
sns.barplot(
    x="ChangeInStock", 
    y="Description", 
    data=top_fast, 
    palette="Blues_r", 
    ax=ax[0]
)
ax[0].set_title("Fast-Moving Products (Stock Drop)")
ax[0].set_xlabel("Change in Stock (units)")
ax[0].set_ylabel("Product Description")

# Slow-Moving / Overstocked Items (stock increased)
sns.barplot(
    x="ChangeInStock", 
    y="Description", 
    data=top_slow, 
    palette="Oranges", 
    ax=ax[1]
)
ax[1].set_title("Slow-Moving / Overstocked Products")
ax[1].set_xlabel("Change in Stock (units)")
ax[1].set_ylabel("Product Description")

plt.tight_layout()
plt.show()


In [ ]:
# Ensure purchase_prices is loaded (if not already)
# purchase_prices = pd.read_csv(f"{base_path}/2017PurchasePricesDec.csv")

# Clean column names for consistency
purchase_prices.columns = purchase_prices.columns.str.strip().str.title()

# Check basic info
purchase_prices.head(2)


In [ ]:
# Group by Brand and calculate average purchase price
brand_prices = (
    purchase_prices.groupby("Brand")["Price"]
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

# Plot Top 10 Brands with visible labels
plt.figure(figsize=(12,6))
sns.barplot(
    data=brand_prices.head(10),
    x="Price",
    y="Brand",
    palette="magma"
)
plt.title("Top 10 Brands by Average Purchase Price (2017)", fontsize=14, weight="bold")
plt.xlabel("Average Purchase Price ($)", fontsize=12)
plt.ylabel("Brand", fontsize=12)

# Make sure labels are fully visible and formatted
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()


In [ ]:
print(f"""
FINAL SUMMARY REPORT — SLOOZE INVENTORY ANALYSIS

 SALES OVERVIEW
• Total Transactions: {len(sales):,}
• Total Revenue: ${sales['Revenue'].sum():,.2f}
• Highest Revenue Product: {sales.groupby('Description')['Revenue'].sum().idxmax()}
• Top Performing Store: {sales.groupby('Store')['Revenue'].sum().idxmax()}

 INVENTORY INSIGHTS
• A-Class Products (High Value): {(abc_df['Category'] == 'A').sum():,}
• B-Class Products (Medium Value): {(abc_df['Category'] == 'B').sum():,}
• C-Class Products (Low Value): {(abc_df['Category'] == 'C').sum():,}
• EOQ (Optimal Order Quantity): {EOQ:,.2f} units
• Reorder Point (ROP): {ROP:,.2f} units
• Average Lead Time: {avg_lead_time:.2f} days

 SUPPLIER PERFORMANCE
• Fastest Supplier: {supplier_lead.iloc[0]['VendorName']} ({supplier_lead.iloc[0]['mean']:.2f} days)
• Slowest Supplier: {supplier_lead.iloc[-1]['VendorName']} ({supplier_lead.iloc[-1]['mean']:.2f} days)

 INVENTORY MOVEMENT
• Fastest Moving Product: {fast_movers.iloc[0]['Description']}
• Slowest Moving Product: {slow_movers.iloc[0]['Description']}

 PURCHASE PRICE ANALYSIS
• Most Expensive Brand (avg): {brand_prices.iloc[0]['Brand']} (${brand_prices.iloc[0]['Price']:.2f})
• Least Expensive Brand (avg): {brand_prices.iloc[-1]['Brand']} (${brand_prices.iloc[-1]['Price']:.2f})

 INSIGHTS SUMMARY
1. Focus inventory control on A-class items — they drive most revenue.
2. Maintain ~{round(ROP)} units reorder level to prevent stockouts.
3. Negotiate better terms with slower suppliers (> {round(supplier_lead.iloc[-1]['mean'])} days lead time).
4. Review slow-moving and high-cost brands for discount or delisting.
5. EOQ ≈ {round(EOQ)} units ensures balanced ordering and holding costs.
""")

Conclusion and Recommendations

Based on the comprehensive analysis:

Key Takeaways
- A-class products (1,126 SKUs) generate ~80% of total revenue.
- Optimal order quantity (EOQ): 11,070 units , balances ordering and holding costs.
- Reorder point (ROP): 51,476 units, ensures stockouts are avoided with average 7.6-day lead time.
- Supplier efficiency varies between 5–13 days; priority vendors include Truett Hurst and Highland Wine Merchants LLC.
- Slowest vendors (e.g. Flavor Essence Inc) should be reviewed for contract optimization.
- Overstocked SKUs should be discounted or ordered less frequently.

Business Recommendations
1. Maintain tighter control on A-class SKUs.
2. Negotiate lead time reductions with slow suppliers.
3. Apply EOQ + ROP thresholds for automated reorder triggers.
4. Introduce periodic ABC reviews every quarter.
5. Explore predictive demand forecasting with ARIMA/Prophet for future iterations.
